In [99]:
import pandas as pd
import numpy as np
import re

In [100]:
df = pd.read_excel('深圳.xlsx')

In [101]:
print(df.columns)

Index(['分区', '小区名称', '所在区域', '房屋户型', '建筑面积', '套内面积', '房屋朝向', '装修情况', '供暖方式',
       '楼层高度', '所在楼层', '户型结构', '建筑类型', '建筑结构', '梯户比例', '配备电梯', '挂牌时间', '上次交易',
       '房屋年限', '抵押信息', '交易权属', '房屋用途', '产权所属', '房本备件', '标题', '总价', '单价', '地区',
       '几室几厅', '房屋面积', '楼层', '建立时间', '户型', '关注人数', '发布日期', 'VR看装修', '房本满五年',
       '近地铁', '房本满两年', '随时看房', '详情页'],
      dtype='object')


In [102]:
# 1. 删除指定的列
columns_to_drop = ['楼层高度', '建筑面积', '楼层', '建立时间', '房本满五年', '房本满两年', '房本备件', '分区', '套内面积', '地区', '几室几厅', '标题', '户型']
df.drop(columns=columns_to_drop, inplace=True)

In [103]:
# 2. 处理 '所在楼层' 列
def process_floor(floor_info):
    if isinstance(floor_info, str):
        match = re.match(r'(.*?)\s*\(共(\d+)层\)', floor_info)
        if match:
            floor_level = match.group(1).strip()  # 楼层高低
            total_floors = match.group(2)  # 总楼层
            return floor_level, total_floors
    return np.nan, np.nan

# 创建新的列 '楼层高低' 和 '楼层总高度'
df[['楼层高低', '楼层总高度']] = df['所在楼层'].apply(lambda x: pd.Series(process_floor(x)))
df.drop(columns=['所在楼层'], inplace=True)

In [104]:
# 3. 填充 '梯户比例' 列中无电梯的行
df['梯户比例'] = df.apply(lambda row: '无电梯' if row['配备电梯'] == '无' else row['梯户比例'], axis=1)

In [105]:
# 4. 处理 '抵押信息' 列，提取‘无抵押’和‘有抵押’
def process_mortgage_info(mortgage_info):
    if isinstance(mortgage_info, str):
        if '无抵押' in mortgage_info:
            return '无抵押'
        elif '有抵押' in mortgage_info:
            return '有抵押'
    return np.nan

df['抵押信息'] = df['抵押信息'].apply(process_mortgage_info)

In [106]:
def clean_housing_data(df):
    import numpy as np

    df['所在区域_clean'] = df['所在区域'].replace({r'\s+': ' '}, regex=True).str.strip()
    
    area_split = df['所在区域_clean'].str.split(' ', expand=True)
    df['区'] = area_split[0] if area_split.shape[1] > 0 else np.nan
    df['街道'] = area_split[1] if area_split.shape[1] > 1 else np.nan
    df['环线位置'] = area_split[2] if area_split.shape[1] > 2 else np.nan
    df.drop(columns=['所在区域_clean'], inplace=True)
    df.drop(columns=['所在区域'], inplace=True)

    layout_split = df['房屋户型'].str.extract(r'((?:\d+室\d+厅))((?:\d+厨\d+卫))')
    df['居住空间'] = layout_split[0]
    df['功能间'] = layout_split[1]
    df.drop(columns=['房屋户型'], inplace=True)

    return df

df = clean_housing_data(df)

In [107]:
# 总价列：去掉“万”并转为 float
df['总价(万元)'] = df['总价'].str.replace('万', '', regex=False).astype(float)
df.drop(columns=['总价'], inplace=True)

# 单价列：去掉“元/平”，去掉逗号，并转为 int
df['单价(元/平)'] = df['单价'].str.replace('元/平', '', regex=False).str.replace(',', '').astype(int)
df.drop(columns=['单价'], inplace=True)

# 房屋面积列：去除“平米”，转为 float
df['房屋面积(平米)'] = df['房屋面积'].str.replace('平米', '', regex=False).astype(float)
df.drop(columns=['房屋面积'], inplace=True)

# 关注人数列：去除“人关注”，转为 int
df['关注人数'] = df['关注人数'].str.replace('人关注', '', regex=False).astype(int)


In [108]:
# 5. 导出处理后的数据
df.to_csv('深圳cleaned_data.csv', index=False)

In [109]:
# 打印处理结果查看
print(df.head())

   小区名称   房屋朝向 装修情况  供暖方式 户型结构  建筑类型  建筑结构  梯户比例 配备电梯       挂牌时间  ... 楼层高低  \
0  兴业大厦      南   其他   NaN   平层    塔楼  钢混结构  两梯八户    有 2024-12-15  ...  高楼层   
1  云峰花园  东南 东北   简装   NaN   平层    塔楼  钢混结构  三梯九户    有 2025-02-22  ...  中楼层   
2  瑞丰大厦     西南   其他   NaN   平层  板塔结合  框架结构  两梯八户    有 2024-09-04  ...  中楼层   
3  云峰花园     西南   精装   NaN   平层    塔楼  钢混结构  三梯九户    有 2025-03-06  ...  高楼层   
4   经贸楼    南 北   其他   NaN   平层    板楼  砖混结构  两梯五户    有 2024-06-21  ...  中楼层   

  楼层总高度    区   街道  环线位置  居住空间   功能间 总价(万元) 单价(元/平) 房屋面积(平米)  
0    19  罗湖区  春风路  None  4室2厅  2厨2卫  315.0   35032    89.92  
1    33  罗湖区  春风路  None  4室2厅  1厨2卫  315.0   35554    88.60  
2    20  罗湖区  春风路  None  4室1厅  1厨1卫  420.0   58799    71.43  
3    33  罗湖区  春风路  None  3室2厅  1厨1卫  350.0   40496    86.43  
4     6  罗湖区  春风路  None  2室1厅  1厨1卫  350.0   43937    79.66  

[5 rows x 32 columns]
